<a href="https://colab.research.google.com/github/royn5618/Talks_Resources/blob/main/PyConPortugal2022/Improved_Keras_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**About:**

This notebook has an improved implmentation of an NLP classifier that predicts emotions using Keras Tuner with some other data and text based approaches for model improvement.

**Data Source on Kaggle:**

https://www.kaggle.com/praveengovi/emotions-dataset-for-nlp

**Data Source on HuggingFace:**

https://huggingface.co/datasets/emotion

# Data import

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_data = pd.read_csv('Data/train.txt', sep=';', names=['text', 'emotion'])
train_data.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [4]:
test_data = pd.read_csv('Data/test.txt', sep=';', names=['text', 'emotion'])
test_data.head()

,text,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [5]:
val_data = pd.read_csv('Data/val.txt', sep=';', names=['text', 'emotion'])
val_data.head()

,text,emotion
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


# Data preparation
## Label Encoding

In [6]:
train_data["emotion"] = train_data["emotion"].astype('category')
train_data["emotion_label"] = train_data["emotion"].cat.codes
train_data.head()

,text,emotion,emotion_label
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0


In [7]:
test_data["emotion"] = test_data["emotion"].astype('category')
test_data["emotion_label"] = test_data["emotion"].cat.codes
test_data.head()

,text,emotion,emotion_label
0,im feeling rather rotten so im not very ambiti...,sadness,4
1,im updating my blog because i feel shitty,sadness,4
2,i never make her separate from me because i do...,sadness,4
3,i left with my bouquet of red and yellow tulip...,joy,2
4,i was feeling a little vain when i did this one,sadness,4


In [8]:
val_data["emotion"] = val_data["emotion"].astype('category')
val_data["emotion_label"] = val_data["emotion"].cat.codes
val_data.head()

,text,emotion,emotion_label
0,im feeling quite sad and sorry for myself but ...,sadness,4
1,i feel like i am still looking at a blank canv...,sadness,4
2,i feel like a faithful servant,love,3
3,i am just feeling cranky and blue,anger,0
4,i can have for a treat or if i am feeling festive,joy,2


## One Hot Encoding

In [9]:
import tensorflow as tf

In [10]:
train_features, train_labels = train_data['text'], tf.one_hot(train_data["emotion_label"], 6)
test_features, test_labels = test_data['text'], tf.one_hot(test_data["emotion_label"], 6)
val_features, val_labels = val_data['text'], tf.one_hot(val_data["emotion_label"], 6)

In [11]:
train_features[:5]

0                              i didnt feel humiliated
1    i can go from feeling so hopeless to so damned...
2     im grabbing a minute to post i feel greedy wrong
3    i am ever feeling nostalgic about the fireplac...
4                                 i am feeling grouchy
Name: text, dtype: object

In [12]:
train_labels[:5]

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)>

## Decoder

In [13]:
def get_labels_from_oh_code(oh_code):
    """ Takes in one-hot encoded matrix
    Returns a list of decoded categories"""
    label_code = np.argmax(oh_code, axis=1)
#     print(label_code)
    label = test_data.emotion.cat.categories[label_code]
#     print(list(label))
    return list(label)

In [14]:
"Test Method"
test= np.array(train_labels[:5])
get_labels_from_oh_code(test)

['sadness', 'sadness', 'anger', 'love', 'anger']

## Text Preprocessing

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

STOPWORDS = stopwords.words('english')
PORTER_STEMMER = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
def preprocess_text(text):
    filtered_text = []
    for each_word in word_tokenize(text):
        if each_word not in STOPWORDS:
            filtered_text.append(PORTER_STEMMER.stem(each_word))
    return " ".join(filtered_text)

In [17]:
train_data['text'] = train_data.text.apply(preprocess_text)
test_data['text'] = test_data.text.apply(preprocess_text)
val_data['text'] = val_data.text.apply(preprocess_text)

In [18]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
vocab_size = 10000
max_seq_len = 20

tokenizer = Tokenizer(oov_token = "<OOV>", num_words=vocab_size, lower=True)
tokenizer.fit_on_texts(train_data['text'])

sequences_train = tokenizer.texts_to_sequences(train_data['text'])
sequences_test = tokenizer.texts_to_sequences(test_data['text'])
sequences_val = tokenizer.texts_to_sequences(val_data['text'])

padded_train = pad_sequences(sequences_train, padding = 'post', maxlen=max_seq_len)
padded_test = pad_sequences(sequences_test, padding = 'post', maxlen=max_seq_len)
padded_val = pad_sequences(sequences_val, padding = 'post', maxlen=max_seq_len)

# Model Designing

## Build a Hyperband Model

In [20]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM

In [21]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 31.3 MB/s 
     |████████████████████████████████| 1.6 MB 57.8 MB/s 


In [22]:
import keras_tuner as kt

In [23]:
vector_size = 300

def model_builder(hp):
    model = Sequential()
    hp_vector_size = hp.Int('vector_size', min_value=100, max_value=500, step=100)
    model.add(
        Embedding(input_dim=vocab_size,
                  output_dim=vector_size,
                  input_length=max_seq_len))
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.6, max_value=0.9, step=0.1)
    model.add(Dropout(hp_dropout_rate))
    hp_lstm_units1 = hp.Int('lstm_units1', min_value=32, max_value=512, step=32)
    model.add(LSTM(hp_lstm_units1,return_sequences=True))
    hp_lstm_units2 = hp.Int('lstm_units2', min_value=16, max_value=512, step=32)
    model.add(LSTM(hp_lstm_units2))
    model.add(Dense(6,activation='softmax'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
    return model

In [24]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_recall", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory="model_trials_1",
                     project_name="emotion_detector_1"
                     )

In [25]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              mode='min', 
                                              patience=5)

In [ ]:
tuner.search(padded_train, 
             train_labels, 
             epochs=20, 
             validation_data=(padded_val, val_labels), 
             callbacks=[stop_early]
             )

Trial 2 Complete [00h 00m 19s]
val_recall: 0.8705000281333923

Best val_recall So Far: 0.8705000281333923
Total elapsed time: 00h 00m 50s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
400               |500               |vector_size
0.7               |0.8               |dropout_rate
384               |320               |lstm_units1
464               |176               |lstm_units2
0.01              |0.001             |learning_rate
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_hps.get('vector_size')

In [ ]:
best_hps.get('dropout_rate')

## Build Model with Best HyperParameters

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model_best_hp = tuner.hypermodel.build(best_hps)

In [ ]:
model_best_hp

### Set Callbacks

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode='min',
                                  patience=5,
                                  verbose=1,
                                  restore_best_weights=True)  
]

## Compile Model

In [ ]:
# No need of compiling

## Train Model

In [ ]:
history = model_best_hp.fit(padded_train, 
                            train_labels, 
                            epochs=20,
                            callbacks=callbacks,
                            validation_data=(padded_val, val_labels))

Epoch 1/10
334/335 [============================>.] - ETA: 0s - loss: 1.5524 - accuracy: 0.3903
Epoch 1: val_loss improved from inf to 1.35542, saving model to models/best_model.h5
335/335 [==============================] - 13s 14ms/step - loss: 1.5520 - accuracy: 0.3909 - val_loss: 1.3554 - val_accuracy: 0.5253
Epoch 2/10
333/335 [============================>.] - ETA: 0s - loss: 1.1010 - accuracy: 0.6254
Epoch 2: val_loss improved from 1.35542 to 0.98002, saving model to models/best_model.h5
335/335 [==============================] - 5s 15ms/step - loss: 1.1001 - accuracy: 0.6258 - val_loss: 0.9800 - val_accuracy: 0.6584
Epoch 3/10
330/335 [============================>.] - ETA: 0s - loss: 0.7386 - accuracy: 0.7630
Epoch 3: val_loss improved from 0.98002 to 0.81094, saving model to models/best_model.h5
335/335 [==============================] - 4s 11ms/step - loss: 0.7400 - accuracy: 0.7629 - val_loss: 0.8109 - val_accuracy: 0.7527
Epoch 4/10
333/335 [============================>.] 

## Visualize and verify the Loss per epoch

In [ ]:
from plotly.graph_objs import *

In [ ]:
metric_to_plot = "loss"
epochs = list(range(1, max(history.epoch) + 2))
training_loss = history.history[metric_to_plot]
validation_loss = history.history["val_" + metric_to_plot]

trace1 = {
    "mode": "lines+markers",
    "name": "Training Loss",
    "type": "scatter",
    "x": epochs,
    "y": training_loss
}

trace2 = {
    "mode": "lines+markers",
    "name": "Validation Loss",
    "type": "scatter",
    "x": epochs,
    "y": validation_loss
}

data = Data([trace1, trace2])
layout = {
    "title": "Training - Validation Loss",
    "xaxis": {
        "title": "Number of epochs",
        "titlefont": {
            "size": 18,
            "color": "#7f7f7f"
        }
    },
    "yaxis": {
        "title": "Loss",
        "titlefont": {
            "size": 18,
            "color": "#7f7f7f"
        }
    }
}
fig = Figure(data=data, layout=layout)
fig.update_layout(hovermode="x unified")
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning: plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.

  DeprecationWarning,


## Visualize and verify the metric per epoch

In [ ]:
metric_to_plot = "accuracy"
epochs = list(range(1, max(history.epoch) + 2))
training_loss = history.history[metric_to_plot]
validation_loss = history.history["val_" + metric_to_plot]

trace1 = {
    "mode": "lines+markers",
    "name": "Training Accuracy",
    "type": "scatter",
    "x": epochs,
    "y": training_loss
}

trace2 = {
    "mode": "lines+markers",
    "name": "Validation Accuracy",
    "type": "scatter",
    "x": epochs,
    "y": validation_loss
}

data = Data([trace1, trace2])
layout = {
    "title": "Training - Validation Accuracy",
    "xaxis": {
        "title": "Number of epochs",
        "titlefont": {
            "size": 18,
            "color": "#7f7f7f"
        }
    },
    "yaxis": {
        "title": "Accuracy",
        "titlefont": {
            "size": 18,
            "color": "#7f7f7f"
        }
    }
}
fig = Figure(data=data, layout=layout)
fig.update_layout(hovermode="x unified")
fig.show()

# Model Evaluation

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# model variable still holds the best model but 
# you can also reload a saved model like this
best_model = keras.models.load_model('models/best_model.h5')

In [ ]:
y_pred_one_hot_encoded = (best_model.predict(padded_train)> 0.5).astype("int32")
y_pred_one_hot_encoded

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=int32)

In [ ]:
y_pred = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))
print(classification_report(train_data['emotion_label'], y_pred))

              precision    recall  f1-score   support

           0       0.78      0.90      0.84      2159
           1       0.93      0.83      0.88      1937
           2       0.94      0.92      0.93      5362
           3       0.80      0.90      0.85      1304
           4       0.95      0.93      0.94      4666
           5       0.85      0.74      0.79       572

    accuracy                           0.90     16000
   macro avg       0.87      0.87      0.87     16000
weighted avg       0.91      0.90      0.90     16000



In [ ]:
# Model Evaluation on Test Data
y_pred_one_hot_encoded = (best_model.predict(padded_test)> 0.5).astype("int32")
y_pred = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))
print(classification_report(test_data['emotion_label'], y_pred))

              precision    recall  f1-score   support

           0       0.64      0.82      0.72       275
           1       0.83      0.70      0.76       224
           2       0.85      0.84      0.84       695
           3       0.62      0.72      0.67       159
           4       0.89      0.83      0.86       581
           5       0.65      0.53      0.58        66

    accuracy                           0.80      2000
   macro avg       0.75      0.74      0.74      2000
weighted avg       0.81      0.80      0.80      2000

